In [2]:
import csv
import tifffile as tiff
import numpy as np
import os
from scipy.interpolate import UnivariateSpline
import numpy as np

In [12]:
def get_list_from_csv(csv_file_path, parse_float=True, skip_header=False):
    """Given a CSV file, converts it to list"""
    def _parse_float_array(arr):
        return [float(item) for item in arr]

    with open(csv_file_path, 'r') as f:
        reader = csv.reader(f)
        csv_list = list(reader)

    parsed_list = csv_list

    if parse_float:
        parsed_list = [_parse_float_array(item) for item in csv_list]

    return parsed_list[1:] if skip_header else parsed_list

def plot_csv_on_rgb_tif(centroids, reference_img_path, tif_output_path, color=[10000, 0, 0], dtype=np.uint16):
    """Given a CSV file, plots the co-ordinates in the CSV on a RGB TIF stack"""
    def _parse_int_array(arr):
        return [int(item) for item in arr]

    def _draw_square(image, coord, size=2):
        coord = _parse_int_array(coord)
        shape_z, shape_y, shape_x, _ = image.shape
        z_range = range(max(0, coord[0]-size), min(shape_z, coord[0]+size))
        y_range = range(max(0, coord[1]-size), min(shape_y, coord[1]+size))
        x_range = range(max(0, coord[2]-size), min(shape_x, coord[2]+size))

        for z in z_range:
            for y in y_range:
                for x in x_range:
                    image[z, y, x, :] = color

        return image

    img = tiff.imread(reference_img_path)


    if img.ndim == 3:
        shape_z, shape_y, shape_x = img.shape
        new_img = np.zeros((shape_z, shape_y, shape_x, 3))
        new_img[:, :, :, 0] = img
        new_img[:, :, :, 1] = img
        new_img[:, :, :, 2] = img
    elif img.ndim == 4:
        shape_z, shape_y, shape_x, _ = img.shape
        new_img = img
  
    for i, c in enumerate(centroids):
        new_img = _draw_square(new_img, c)

    tiff.imsave(tif_output_path, new_img.astype(dtype))

In [17]:
img = tiff.imread('lids/lids_smaller_stack.tif')
labels = get_list_from_csv('lids/labels.csv')
predictions = get_list_from_csv('lids/predictions.csv')
plot_csv_on_rgb_tif(labels, 'lids/lids_smaller_stack.tif', 'lids/marked_final.tif', color=[0, img.max(), 0])
plot_csv_on_rgb_tif(predictions, 'lids/marked_final.tif', 'lids/marked_final.tif', color=[img.max(), 0, 0])